In [1]:
import numpy as np 
import pandas as pd
import tensorflow as tf
import tensorflow.keras.layers as L
import matplotlib.pyplot as plt
import plotly.graph_objects as go
import plotly.express as px
from sklearn.model_selection import train_test_split
from keras.models import load_model
import joblib
from tensorflow.keras import models, layers

2022-04-09 13:37:26.861739: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-04-09 13:37:26.861809: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [2]:
data = pd.read_csv('../raw_data/age_gender.csv')

## Converting pixels into numpy array
data['pixels']=data['pixels'].apply(lambda x:  np.array(x.split(), dtype="float32"))

data.head()

,age,ethnicity,gender,img_name,pixels
0,1,2,0,20161219203650636.jpg.chip.jpg,"[129.0, 128.0, 128.0, 126.0, 127.0, 130.0, 133..."
1,1,2,0,20161219222752047.jpg.chip.jpg,"[164.0, 74.0, 111.0, 168.0, 169.0, 171.0, 175...."
2,1,2,0,20161219222832191.jpg.chip.jpg,"[67.0, 70.0, 71.0, 70.0, 69.0, 67.0, 70.0, 79...."
3,1,2,0,20161220144911423.jpg.chip.jpg,"[193.0, 197.0, 198.0, 200.0, 199.0, 200.0, 202..."
4,1,2,0,20161220144914327.jpg.chip.jpg,"[202.0, 205.0, 209.0, 210.0, 209.0, 209.0, 210..."


In [3]:
print('Total rows: {}'.format(len(data)))
print('Total columns: {}'.format(len(data.columns)))

Total rows: 23705
Total columns: 5


In [4]:
## normalizing pixels data
data['pixels'] = data['pixels'].apply(lambda x: x/255)

## calculating distributions
age_dist = data['age'].value_counts()
ethnicity_dist = data['ethnicity'].value_counts()
gender_dist = data['gender'].value_counts().rename(index={0:'Male',1:'Female'})

def ditribution_plot(x,y,name):
    fig = go.Figure([
        go.Bar(x=x, y=y)
    ])

    fig.update_layout(title_text=name)
    fig.show()

In [ ]:
ditribution_plot(x=age_dist.index, y=age_dist.values, name='Age Distribution')

In [ ]:
ditribution_plot(x=ethnicity_dist.index, y=ethnicity_dist.values, name='Ethnicity Distribution')

In [ ]:
ditribution_plot(x=gender_dist.index, y=gender_dist.values, name='Gender Distribution')

In [5]:
X = np.array(data['pixels'].tolist())

## Converting pixels from 1D to 3D
X = X.reshape(X.shape[0],48,48,1)

In [ ]:
plt.figure(figsize=(16,16))
for i in range(1480,1500):
    plt.subplot(5,5,(i%25)+1)
    plt.xticks([])
    plt.yticks([])
    plt.grid(False)
    plt.imshow(data['pixels'].iloc[i].reshape(48,48))
    plt.xlabel(
        "Age:"+str(data['age'].iloc[i])+
        "  Ethnicity:"+str(data['ethnicity'].iloc[i])+
        "  Gender:"+ str(data['gender'].iloc[i])
    )
plt.show()

In [ ]:
y_gender = np.array(data['gender'])

X_train_gender, X_test_gender, y_train_gender, y_test_gender = train_test_split(
    X, y_gender, test_size=0.22, random_state=37
)

In [7]:
'''
model_gender = tf.keras.Sequential([
    L.InputLayer(input_shape=(48,48,1)),
    L.Conv2D(32, (3, 3), activation='relu', input_shape=(32, 32, 3)),
    L.BatchNormalization(),
    L.MaxPooling2D((2, 2)),
    L.Conv2D(64, (3, 3), activation='relu'),
    L.MaxPooling2D((2, 2)),
    L.Flatten(),
    L.Dense(64, activation='relu'),
    L.Dropout(rate=0.5),
    L.Dense(1, activation='sigmoid')
])
'''
def initialize_model():
    model = models.Sequential()
    model.add(layers.Conv2D(32, (3,3), input_shape=(48, 48, 1), activation="relu"))
    model.add(layers.BatchNormalization())
    model.add(layers.MaxPool2D(pool_size=(2,2))) 
    model.add(layers.Conv2D(64, (3,3), activation="relu"))
    model.add(layers.MaxPool2D(pool_size=(2,2)))

    model.add(layers.Flatten())

    model.add(layers.Dense(64, activation='relu'))
    model.add(layers.Dropout(rate=0.5))
    model.add(layers.Dense(1, activation='sigmoid'))
    return model

model_gender = initialize_model()

'''
model_gender.compile(optimizer='sgd',
              loss=tf.keras.losses.BinaryCrossentropy(),
              metrics=['accuracy'])


## Stop training when validation loss reach 0.2700
class myCallback(tf.keras.callbacks.Callback):
    def on_epoch_end(self, epoch, logs={}):
        if(logs.get('val_loss')<0.2900):
            print("\nReached 0.2900 val_loss so cancelling training!")
            self.model.stop_training = True
        
callback = myCallback()

model_gender.summary()
'''

2022-04-09 13:37:49.723459: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2022-04-09 13:37:49.723534: W tensorflow/stream_executor/cuda/cuda_driver.cc:269] failed call to cuInit: UNKNOWN ERROR (303)
2022-04-09 13:37:49.723569: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (DESKTOP-BKG5TOC): /proc/driver/nvidia/version does not exist
2022-04-09 13:37:49.723846: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


'\nmodel_gender.compile(optimizer=\'sgd\',\n              loss=tf.keras.losses.BinaryCrossentropy(),\n              metrics=[\'accuracy\'])\n\n\n## Stop training when validation loss reach 0.2700\nclass myCallback(tf.keras.callbacks.Callback):\n    def on_epoch_end(self, epoch, logs={}):\n        if(logs.get(\'val_loss\')<0.2900):\n            print("\nReached 0.2900 val_loss so cancelling training!")\n            self.model.stop_training = True\n        \ncallback = myCallback()\n\nmodel_gender.summary()\n'

In [8]:
from tensorflow.keras.optimizers import RMSprop,SGD,Adam
from keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau

checkpoint = ModelCheckpoint("model_gender.h5", monitor='val_acc', verbose=1, save_best_only=True, mode='max')

early_stopping = EarlyStopping(monitor='val_loss',
                          min_delta=0,
                          patience=3,
                          verbose=1,
                          restore_best_weights=True
                          )

reduce_learningrate = ReduceLROnPlateau(monitor='val_loss',
                              factor=0.2,
                              patience=3,
                              verbose=1,
                              min_delta=0.0001)

callbacks_list = [early_stopping,checkpoint,reduce_learningrate]

epochs = 50

model_gender.compile(loss='binary_crossentropy',
              optimizer = 'sgd',
              metrics=['accuracy'])

In [ ]:
history_gender = model_gender.fit(
    X_train_gender, y_train_gender, epochs=40, validation_split=0.1, batch_size=64, callbacks=callbacks_list
)

In [ ]:
joblib.dump(model_gender, 'model_gender.joblib')

In [ ]:
model_gender.save('model_gender.h5')

In [ ]:
fig = px.line(
    history_gender.history, y=['loss', 'val_loss'],
    labels={'index': 'epoch', 'value': 'loss'}, 
    title='Training History')
fig.show()

In [ ]:
loss, acc = model_gender.evaluate(X_test_gender,y_test_gender,verbose=0)
print('Test loss: {}'.format(loss))
print('Test Accuracy: {}'.format(acc))

Ethiny

In [9]:
from tensorflow.keras.utils import to_categorical 

y_ethiny = data['ethnicity']
ethnicity = to_categorical(y_ethiny, num_classes = 5)

X_train_ethiny, X_test_ethiny, y_train_ethiny, y_test_ethiny = train_test_split(
    X, ethnicity, test_size=0.22, random_state=37
)

In [ ]:

def initialize_model():
    model = models.Sequential()
    model.add(layers.Conv2D(32, (3,3), input_shape=(48, 48, 1), activation="relu"))
    model.add(layers.BatchNormalization())
    model.add(layers.MaxPool2D(pool_size=(2,2))) 
    model.add(layers.Conv2D(64, (3,3), activation="relu"))
    model.add(layers.MaxPool2D(pool_size=(2,2)))

    model.add(layers.Flatten())

    model.add(layers.Dense(64, activation='relu'))
    model.add(layers.Dropout(rate=0.5))
    model.add(layers.Dense(5, activation = 'softmax'))
    return model

model_ethiny = initialize_model()


In [ ]:
model_ethiny.compile(loss='categorical_crossentropy',
              optimizer = 'rmsprop',
              metrics=['accuracy'])

In [ ]:
history_ethiny = model_ethiny.fit(
    X_train_ethiny, y_train_ethiny, epochs=20, validation_split=0.1, batch_size=64, callbacks=callbacks_list
)

In [ ]:
model_ethiny.save("model_ethiny.h5")

In [ ]:
loss, acc = model_ethiny.evaluate(X_test_ethiny,y_test_ethiny,verbose=0)
print('Test loss: {}'.format(loss))
print('Test Accuracy: {}'.format(acc))

Age

In [10]:
def ages(x):
    if x<=2:
        return 0
    elif x<=12:
        return 1
    elif x<=18:
        return 2
    elif x<= 26:
        return 3
    elif x<= 50:
        return 4
    else:
        return 5

In [11]:
data["age_new"] = data["age"].apply(lambda x: ages(x))

In [12]:
age_cat = to_categorical(data['age_new'])

In [14]:
X_train_age, X_test_age, y_train_age, y_test_age = train_test_split(
    X, age_cat, test_size=0.22, random_state=37
)

In [15]:

def initialize_model():
    model = models.Sequential()
    model.add(layers.Conv2D(32, (3,3), input_shape=(48, 48, 1), activation="relu"))
    model.add(layers.BatchNormalization())
    model.add(layers.MaxPool2D(pool_size=(2,2))) 
    model.add(layers.Conv2D(64, (3,3), activation="relu"))
    model.add(layers.MaxPool2D(pool_size=(2,2)))

    model.add(layers.Flatten())

    model.add(layers.Dense(64, activation='relu'))
    model.add(layers.Dropout(rate=0.5))
    model.add(layers.Dense(6, activation = 'softmax'))
    return model

model_age = initialize_model()

In [16]:
model_age.compile(loss='categorical_crossentropy',
              optimizer = 'rmsprop',
              metrics=['accuracy'])

In [17]:
history_age = model_age.fit(
    X_train_age, y_train_age, epochs=20, validation_split=0.1, batch_size=64, callbacks=callbacks_list
)

Epoch 1/20
260/260 [==============================] - 35s 129ms/step - loss: 1.3349 - accuracy: 0.4475 - val_loss: 1.5848 - val_accuracy: 0.4619 - lr: 0.0010
Epoch 2/20
260/260 [==============================] - 33s 128ms/step - loss: 1.0859 - accuracy: 0.5379 - val_loss: 1.1936 - val_accuracy: 0.5635 - lr: 0.0010
Epoch 3/20
260/260 [==============================] - 33s 126ms/step - loss: 1.0129 - accuracy: 0.5659 - val_loss: 1.0739 - val_accuracy: 0.5646 - lr: 0.0010
Epoch 4/20
260/260 [==============================] - 33s 126ms/step - loss: 0.9525 - accuracy: 0.5951 - val_loss: 0.8473 - val_accuracy: 0.6636 - lr: 0.0010
Epoch 5/20
260/260 [==============================] - 33s 128ms/step - loss: 0.9114 - accuracy: 0.6096 - val_loss: 0.8500 - val_accuracy: 0.6387 - lr: 0.0010
Epoch 6/20
260/260 [==============================] - 34s 132ms/step - loss: 0.8659 - accuracy: 0.6262 - val_loss: 1.1016 - val_accuracy: 0.5998 - lr: 0.0010
Epoch 7/20
260/260 [==============================] 

In [18]:
model_age.save("model_age.h5")